In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_8_day()
df["p-pet"] = df["precip"] - df["PET"]
df["p-et"] = df["precip"] - df["wp_le"] * .408
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN,p-pet,p-et
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.396594,0.353988,0.641934,31.567899,17.204530,29.105844,24.562860,29.105844,-7.33,-5.566712
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.407211,0.376832,0.654545,29.570000,17.390000,29.760736,26.096551,29.760736,-6.52,-5.507494
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.416158,0.394189,0.664269,31.097908,17.235624,30.299442,25.986418,30.299442,-6.92,-6.115032
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.425106,0.411547,0.673993,30.868718,17.248525,28.978868,25.122209,28.978868,-6.35,-4.633187
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.434053,0.428904,0.683716,30.657792,17.259663,30.653667,27.830367,30.653667,-5.13,-4.464786


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 0 to 1029
Data columns (total 30 columns):
PET             1030 non-null float64
VPD             1030 non-null float64
air_temp        1030 non-null float64
doy             1030 non-null int64
precip          1030 non-null float64
soil_temp       1030 non-null float64
sw_in           1030 non-null float64
wind_speed      1030 non-null float64
year            1030 non-null int64
wp_RNET         1030 non-null float64
wp_ch4_gf       1030 non-null float64
wp_co2_gf       1030 non-null float64
wp_er           1030 non-null float64
wp_gpp          1030 non-null float64
wp_h            1030 non-null float64
wp_le           1030 non-null float64
wp_bnd1         1030 non-null float64
wp_bnd2         1030 non-null float64
wp_bnd3         1030 non-null float64
wp_bnd7         1030 non-null float64
wp_evi          1030 non-null float64
wp_lswi2        1030 non-null float64
wp_ndvi         1030 non-null float64
wp_LST.day      1030 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2", "wp_RNET", "wp_gpp", "p-pet", "p-et"]
X, Y = exp.featurize(df, train_cols, ["wp_ch4_gf"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1030, 16)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.912403436125
CV Scores:  0.900804147618, 0.910942498597, 0.900552542421, 0.934174460093, 0.933460872783, 0.919591724698, 0.903588103569, 0.906948065467, 0.893855619911, 0.92011632609
OOB score: 0.92009354719
Feature Importances:
('wp_evi', 0.19260149695000217)
('wp_lswi2', 0.17331400073983796)
('wp_ndvi', 0.13250351008033617)
('wp_LST.night', 0.10922386654544622)
('wp_gpp', 0.087022760422156509)
('air_temp', 0.054757976452367213)
('wp_LST.day', 0.045155730168932706)
('p-et', 0.043047756403666836)
('wp_RNET', 0.032453476837591963)
('wind_speed', 0.0293745034195491)
('p-pet', 0.028504575517176337)
('sw_in', 0.024775120561778538)
('PET', 0.019003048205215353)
('VPD', 0.015980313436074669)
('LW_IN.wp', 0.01137324462385636)
('precip', 0.00090861963601216757)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.903639143828
CV Scores:  0.89912600702, 0.902062120996, 0.913623065874, 0.922881655011, 0.911402497044, 0.905406354634, 0.880720922689, 0.895411715567, 0.900345117402, 0.905411982047
Feature Importances:
('wp_lswi2', 0.1196434827065867)
('wp_LST.night', 0.11269983631198179)
('wp_gpp', 0.10352795782210351)
('wp_evi', 0.089494024506626815)
('wp_ndvi', 0.087498204712193425)
('wp_LST.day', 0.069295274413217728)
('VPD', 0.064872004218676699)
('wind_speed', 0.057034837248252916)
('p-et', 0.055834987437763814)
('sw_in', 0.045929250827028659)
('air_temp', 0.042453678296073799)
('wp_RNET', 0.040512598191107058)
('LW_IN.wp', 0.038305262440076218)
('PET', 0.035527001859900278)
('p-pet', 0.033294880738045166)
('precip', 0.0040767182703654072)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.119894544778
CV Scores:  0.118900035978, 0.175663506056, 0.135286279469, 0.0638738232421, 0.14358958002, 0.0949091591104, 0.193898877611, 0.126149845873, 0.137464080958, 0.00921025946685


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y, regr=r.dnn(nn_steps=10000, nn_lr=0.2, hidden_units=[50, 50]), k=10)

Running Neural Network Cross Validation...
Step #1, avg. loss: 1578644.50000
Step #1001, epoch #100, avg. loss: 304176.90625
Step #2001, epoch #200, avg. loss: 85640.81250
Step #3001, epoch #300, avg. loss: 46115.69531
Step #4001, epoch #400, avg. loss: 27882.35352
Step #5001, epoch #500, avg. loss: 18918.15430
Step #6001, epoch #600, avg. loss: 13901.17090
Step #7001, epoch #700, avg. loss: 11007.00293
Step #8001, epoch #800, avg. loss: 9121.14648
Step #9001, epoch #900, avg. loss: 7309.74463
Step #1, avg. loss: 1418241.50000
Step #1001, epoch #100, avg. loss: 289625.81250
Step #2001, epoch #200, avg. loss: 83488.54688
Step #3001, epoch #300, avg. loss: 46471.83203
Step #4001, epoch #400, avg. loss: 29278.96484
Step #5001, epoch #500, avg. loss: 19886.13281
Step #6001, epoch #600, avg. loss: 14146.14844
Step #7001, epoch #700, avg. loss: 11041.96094
Step #8001, epoch #800, avg. loss: 8713.40918
Step #9001, epoch #900, avg. loss: 7493.98389
Step #1, avg. loss: 1165988.75000
Step #1001,

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.2,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x10b319f50>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=10000,
          tf_master='', tf_random_seed=42, verbose=1)